In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24410772
paper_name = 'okada_shima_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 71 x 5


In [7]:
original_data.head()

,Systematic name,Standard name,H2O2sensitivity,P-value,Description
0,YPL143W,RPL33A,0.065,1.10E−13,Ribosomal 60S subunit protein L33A
1,YPL235W,RVB2,0.092,5.79E−09,ATP-dependent DNA helicase
2,YLR293C,GSP1,0.126,4.41E−07,RNA processing and transport
3,YDL193W,NUS1,0.133,1.01E−05,Putative prenyltransferase
4,YBL014C,RRN6,0.145,9.00E−07,Component of the core factor (CF) rDNA transcr...


In [8]:
original_data['orf'] = original_data['Systematic name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Systematic name, Standard name, H2O2sensitivity, P-value, Description, orf]
Index: []


In [12]:
original_data['data'] = original_data['H2O2sensitivity']

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(71, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [570]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,570
data_type,value
orf,
YBL014C,0.145
YDL031W,0.466
YDL043C,0.256
YDL058W,0.483
YDL193W,0.133


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,570
,data_type,value
gene_id,orf,
102,YBL014C,0.145
1910,YDL031W,0.466
1921,YDL043C,0.256
1936,YDL058W,0.483
2072,YDL193W,0.133


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id         570          
data_type        value    valuez
gene_id orf                     
102     YBL014C  0.145  2.962367
1910    YDL031W  0.466  9.520434
1921    YDL043C  0.256  5.230110
1936    YDL058W  0.483  9.867746
2072    YDL193W  0.133  2.717205

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24410772...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  4.30it/s]

Updating the data_modified_on field...
